## 0. Simulate some data and fit an emulator

In [1]:
import torch

from autoemulate.experimental.simulations.projectile import ProjectileMultioutput
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)

In [2]:
sim = ProjectileMultioutput()
x = sim.sample_inputs(100)
y = sim.forward_batch(x)

Running simulations:   0%|          | 0/100 [00:00<?, ?it/s]

Running simulations: 100%|██████████| 100/100 [00:00<00:00, 1190.39it/s]

Successfully completed 100/100 simulations (100.0%)


In [3]:
gp = GaussianProcessExact(x, y)
gp.fit(x, y)

## 1. Simple HMC example.

In [4]:
from autoemulate.experimental.calibration.hmc import HMCCalibrator

Lets start with an observation inside the training range, we should be able to recover the input parameters.

In other words, just pick as observation one row of our simulated values.

In [5]:
idx = 4
observations = {
    param: val for param, val in 
    zip(sim.output_names, y[idx, :])
}

In [16]:
print("Inputs: ", x[idx])
print("Outputs: ", y[idx])

Inputs:  tensor([1.4571e-01, 2.8744e+02])
Outputs:  tensor([3.1372, 2.6470], dtype=torch.float64)


In [6]:
# use the simulator parameter_range 
hmc = HMCCalibrator(gp, sim.parameters_range, observations, 1.0)

Run MCMC (note that below we have set the number of MCMC steps to a very low number, don't expect convergence).

In [23]:
mcmc = hmc.run_mcmc(warmup_steps=50, num_samples=200)

Sample: 100%|██████████| 250/250 [07:46,  1.87s/it, step size=1.44e-04, acc. prob=0.274]


The returned Pyro MCMC object has methods for accessing the generated samples (`mcmc.get_samples()`) or, as shown below, to get just their summary statistics.

In [24]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
         c      0.48      0.15      0.50      0.27      0.72      4.41      1.85
        v0    461.84     54.74    484.13    364.89    526.73     24.59      1.00

Number of divergences: 7


In [25]:
mcmc.get_samples()

{'c': tensor([0.4973, 0.5020, 0.5030, 0.5077, 0.5118, 0.4779, 0.4899, 0.4877, 0.5523,
         0.5548, 0.5588, 0.5680, 0.5733, 0.5723, 0.5744, 0.6222, 0.6200, 0.5790,
         0.5935, 0.4949, 0.5256, 0.5346, 0.5497, 0.5776, 0.6061, 0.6270, 0.6969,
         0.7057, 0.6817, 0.5906, 0.6328, 0.6397, 0.5882, 0.6164, 0.6668, 0.7094,
         0.7168, 0.7144, 0.7118, 0.7007, 0.7059, 0.7028, 0.6971, 0.6935, 0.6702,
         0.6709, 0.6706, 0.6616, 0.6592, 0.6540, 0.6508, 0.6613, 0.6582, 0.6244,
         0.4305, 0.4151, 0.4136, 0.4411, 0.4276, 0.5262, 0.5125, 0.4745, 0.4714,
         0.4813, 0.5758, 0.5710, 0.5668, 0.5657, 0.5668, 0.5795, 0.5707, 0.5687,
         0.5088, 0.5386, 0.6977, 0.7210, 0.6428, 0.6358, 0.6391, 0.6078, 0.6218,
         0.6587, 0.6623, 0.6580, 0.6379, 0.6128, 0.6317, 0.6311, 0.6640, 0.6748,
         0.6403, 0.5807, 0.5744, 0.5224, 0.5254, 0.5619, 0.5678, 0.5837, 0.5970,
         0.5844, 0.5092, 0.4979, 0.4684, 0.4584, 0.4595, 0.4484, 0.4535, 0.4607,
         0.4621, 0.5210

In [28]:
hmc.posterior_predictive(mcmc)

tensor([[3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1372, 2.6470],
        [3.1

## 2. Combining this with sensitivity analysis and history matching.

The `HMCCalibrator` object has an option to provide a list of parameters to calibrate. These can be the result of `SensitivityAnalysis`, or just a list provided by the user.

Similarly, the user provides parameter ranges from which to sample or set the parameter values. This can be simply the range of the simulator or one can use `HistoryMatching` to reduce the parameter range and pass that to the `HMCCalibrator` instead. 

Below we demonstrate how to do both.

In [29]:
from autoemulate.experimental.sensitivity_analysis import SensitivityAnalysis
from autoemulate.experimental.calibration.history_matching import HistoryMatching

1. Run sensitivity analysis and get top N parameters (here we just get the top 1).

In [30]:
problem = {
        "num_vars": 2,
        "names": sim.param_names,
        "bounds": sim.param_bounds,
    }
sa = SensitivityAnalysis(gp, problem=problem)
df = sa.run("sobol")

/Users/rjersakova/Documents/Projects/autoemulate/.venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
/Users/rjersakova/Documents/Projects/autoemulate/.venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


Notice that the output is just a list of strings, the user does not need to run SA to set a subset of parameters to calibrate.

In [31]:
top_param = sa.top_n_sobol_params(df, 1)
top_param

['v0']

2. Run history matching and generate new parameter bounds from NROY samples.

In [32]:
# start with some GP predictions
x_new = sim.sample_inputs(20)
output = gp.predict(torch.tensor(x_new, dtype=torch.float32))
pred_means, pred_vars = (
    output.mean.float().detach(),
    output.variance.float().detach(),
)

/var/folders/bj/kdwy1bhj3h728lr5xdj19yd40000gr/T/ipykernel_76391/3687513851.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = gp.predict(torch.tensor(x_new, dtype=torch.float32))


In [33]:
# generate NROY samples
hm = HistoryMatching(
    # add noise to observations
    observations={k: [v, 10.0] for k,v in observations.items()},
    threshold=5.0,
    rank=2
)
implausability = hm.calculate_implausibility(pred_means, pred_vars)
nroy_samples = hm.get_nroy(implausability, x_new)
nroy_samples

tensor([[-5.7056e-01,  3.1014e+02],
        [-9.8348e-01,  9.4139e+02],
        [-9.9788e-02,  4.2735e+02],
        [-2.1315e+00,  2.2844e+01],
        [-1.1696e+00,  7.8298e+02],
        [ 7.7927e-01,  8.2514e+02],
        [-3.5781e-01,  6.5609e+02],
        [-2.8925e+00,  1.2202e+02],
        [ 3.1747e-01,  8.7228e+02],
        [ 5.3147e-01,  3.7946e+02],
        [-4.1902e+00,  5.3759e+01]])

The newly generated range is slightly narrower than the range of the simulator.

In [34]:
# get new param bounds
nroy_param_range = hm.generate_param_bounds(nroy_samples, param_names = sim.param_names)
nroy_param_range

{'c': (-4.438632011413574, 1.0277363061904907),
 'v0': (-23.083009719848633, 987.3121337890625)}

3. Pass results to the HMCCalibrator object.

In [35]:
hmc = HMCCalibrator(
    gp, 
    nroy_param_range, 
    observations, 
    1.0,
    top_param
    )

In [36]:
mcmc = hmc.run_mcmc(warmup_steps=10, num_samples=100)

Sample: 100%|██████████| 110/110 [00:00, 372.58it/s, step size=2.65e+01, acc. prob=0.025]


In [37]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
        v0    -23.08      0.00    -23.08    -23.08    -23.08       nan       nan

Number of divergences: 98
